In [26]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

In [16]:
# Import the data
orders_df = pd.read_csv("amazon_orders.csv")
print(orders_df.head())

# See what countries the data targets
orders_df['Country'].value_counts()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [12]:
# Cleaning the data
orders_df.info()

# Extract useful columns for pattern mining
pattern_mining_df = orders_df.copy(deep=True)[['Row ID', 'Order ID', 'Customer ID', 'Product ID', 'Product Name']]
pattern_mining_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

,Row ID,Order ID,Customer ID,Product ID,Product Name
0,1,CA-2017-152156,CG-12520,FUR-BO-10001798,Bush Somerset Collection Bookcase
1,2,CA-2017-152156,CG-12520,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,..."
2,3,CA-2017-138688,DV-13045,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...
3,4,US-2016-108966,SO-20335,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table
4,5,US-2016-108966,SO-20335,OFF-ST-10000760,Eldon Fold 'N Roll Cart System


In [13]:
# Remove nulls from pattern mining data 
pattern_mining_df.isnull().sum() # No null data present

Row ID          0
Order ID        0
Customer ID     0
Product ID      0
Product Name    0
dtype: int64

In [22]:
# Group our data by orders and transactions, we then some up entries to get the sum of items purchased and
# then unstack them so that invoice id is on row and each product has its own column

basket = pattern_mining_df.copy()
basket = basket.groupby(['Order ID', 'Product ID'])['Product Name'].count().unstack().reset_index().fillna(0).set_index("Order ID")

basket.head()

Product ID,FUR-BO-10000112,FUR-BO-10000330,FUR-BO-10000362,FUR-BO-10000468,FUR-BO-10000711,FUR-BO-10000780,FUR-BO-10001337,FUR-BO-10001519,FUR-BO-10001567,FUR-BO-10001601,...,TEC-PH-10004833,TEC-PH-10004875,TEC-PH-10004896,TEC-PH-10004897,TEC-PH-10004908,TEC-PH-10004912,TEC-PH-10004922,TEC-PH-10004924,TEC-PH-10004959,TEC-PH-10004977
Order ID,,,,,,,,,,,,,,,,,,,,,
CA-2015-100006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2015-100090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2015-100293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2015-100328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA-2015-100363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# We then set any values above zero to 1 to indicate that it was purchased and values that are 0 to 0 to indicate
# that they were not purchased
def is_purchased(x):
    if x <= 0:
        return 0
    if x > 0:
        return 1
    
basket_encoded = basket.map(is_purchased)
basket_encoded.head()

Product ID,FUR-BO-10000112,FUR-BO-10000330,FUR-BO-10000362,FUR-BO-10000468,FUR-BO-10000711,FUR-BO-10000780,FUR-BO-10001337,FUR-BO-10001519,FUR-BO-10001567,FUR-BO-10001601,...,TEC-PH-10004833,TEC-PH-10004875,TEC-PH-10004896,TEC-PH-10004897,TEC-PH-10004908,TEC-PH-10004912,TEC-PH-10004922,TEC-PH-10004924,TEC-PH-10004959,TEC-PH-10004977
Order ID,,,,,,,,,,,,,,,,,,,,,
CA-2015-100006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Remove orders that only have one item since we want to analyze items that often come together
basket_filtered = basket_encoded[(basket_encoded > 0).sum(axis=1) >= 2] # sum every column that has a value greater than 0 and return those where value is greter than or equal 2

basket_filtered.head()

Product ID,FUR-BO-10000112,FUR-BO-10000330,FUR-BO-10000362,FUR-BO-10000468,FUR-BO-10000711,FUR-BO-10000780,FUR-BO-10001337,FUR-BO-10001519,FUR-BO-10001567,FUR-BO-10001601,...,TEC-PH-10004833,TEC-PH-10004875,TEC-PH-10004896,TEC-PH-10004897,TEC-PH-10004908,TEC-PH-10004912,TEC-PH-10004922,TEC-PH-10004924,TEC-PH-10004959,TEC-PH-10004977
Order ID,,,,,,,,,,,,,,,,,,,,,
CA-2015-100090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CA-2015-100762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# We will now use Apriori algorithm to find the common occuring patterns